In [1]:
from pypokerengine.players import BasePokerPlayer
from pypokerengine.utils.card_utils import gen_cards, estimate_hole_card_win_rate, _montecarlo_simulation
from pypokerengine.api.game import setup_config, start_poker

import random as rand

## CallPlayer

In [2]:
class CallPlayer(BasePokerPlayer):  # Do not forget to make parent class as "BasePokerPlayer"

    #  we define the logic to make an action through this method. (so this method would be the core of your AI)
    def declare_action(self, valid_actions, hole_card, round_state):
        # valid_actions format => [raise_action_info, call_action_info, fold_action_info]
        call_action_info = valid_actions[1]
        action, amount = call_action_info["action"], call_action_info["amount"]
        return action, amount   # action returned here is sent to the poker engine

    def receive_game_start_message(self, game_info):
        pass

    def receive_round_start_message(self, round_count, hole_card, seats):
        pass

    def receive_street_start_message(self, street, round_state):
        pass

    def receive_game_update_message(self, action, round_state):
        pass

    def receive_round_result_message(self, winners, hand_info, round_state):
        pass

## FoldPlayer

In [3]:
class FoldPlayer(BasePokerPlayer):  # Do not forget to make parent class as "BasePokerPlayer"
    #  we define the logic to make an action through this method. (so this method would be the core of your AI)
    def declare_action(self, valid_actions, hole_card, round_state):
        # valid_actions format => [raise_action_info, call_action_info, fold_action_info]
        call_action_info = valid_actions[0]
        action, amount = call_action_info["action"], call_action_info["amount"]
        return action, amount   # action returned here is sent to the poker engine

    def receive_game_start_message(self, game_info):
        pass

    def receive_round_start_message(self, round_count, hole_card, seats):
        pass

    def receive_street_start_message(self, street, round_state):
        pass

    def receive_game_update_message(self, action, round_state):
        pass

    def receive_round_result_message(self, winners, hand_info, round_state):
        pass

## HeuristicPlayer

In [4]:
class HeuristicPlayer(BasePokerPlayer):  # Do not forget to make parent class as "BasePokerPlayer"
    #  we define the logic to make an action through this method. (so this method would be the core of your AI)
    def declare_action(self, valid_actions, hole_card, round_state):
        # valid_actions format => [raise_action_info, call_action_info, fold_action_info]
        call_action_info = valid_actions[1]
        fold_action_info = valid_actions[0]

        community_card = round_state['community_card']
        win_rate = estimate_hole_card_win_rate(nb_simulation=150, nb_player=self.nb_player,
                                               hole_card=gen_cards(hole_card),
                                               community_card=gen_cards(community_card))
        if win_rate > 1 / float(self.nb_player) + 0.1:
            action, amount = call_action_info["action"], call_action_info["amount"]
        else:
            action, amount = fold_action_info["action"], fold_action_info["amount"]
        return action, amount

    def receive_game_start_message(self, game_info):
        self.nb_player = game_info['player_num']

    def receive_round_start_message(self, round_count, hole_card, seats):
        pass

    def receive_street_start_message(self, street, round_state):
        pass

    def receive_game_update_message(self, action, round_state):
        pass

    def receive_round_result_message(self, winners, hand_info, round_state):
        pass

## RandomPlayer

In [5]:
class RandomPlayer(BasePokerPlayer):
    def __init__(self):
        self.fold_ratio, self.call_ratio, raise_ratio = 1.0/5, 3.0/5, 1.0/5

    def set_action_ratio(self, fold_ratio, call_ratio, raise_ratio):
        ratio = [fold_ratio, call_ratio, raise_ratio]
        scaled_ratio = [ 1.0 * num / sum(ratio) for num in ratio]
        self.fold_ratio, self.call_ratio, self.raise_ratio = scaled_ratio

    def declare_action(self, valid_actions, hole_card, round_state):
        choice = self.__choice_action(valid_actions)
        action = choice["action"]
        amount = choice["amount"]
        if action == "raise":
            amount = rand.randrange(amount["min"], max(amount["min"], amount["max"]) + 1)
        return action, amount

    def __choice_action(self, valid_actions):
        r = rand.random()
        if r <= self.fold_ratio:
            return valid_actions[0]
        elif r <= self.call_ratio:
            return valid_actions[1]
        else:
            return valid_actions[2]


    def receive_game_start_message(self, game_info):
        pass

    def receive_round_start_message(self, round_count, hole_card, seats):
        pass

    def receive_street_start_message(self, street, round_state):
        pass

    def receive_game_update_message(self, new_action, round_state):
        pass

    def receive_round_result_message(self, winners, hand_info, round_state):
        pass

## Simulations

In [6]:
config = setup_config(max_round=2, initial_stack=1500, small_blind_amount=15)
config.register_player(name="CallPlayer1", algorithm=CallPlayer())
config.register_player(name="CallPlayer2", algorithm=CallPlayer())
config.register_player(name="CallPlayer3", algorithm=CallPlayer())

config.register_player(name="FoldPlayer1", algorithm=FoldPlayer())
config.register_player(name="FoldPlayer2", algorithm=FoldPlayer())

config.register_player(name="HeuristicPlayer1", algorithm=HeuristicPlayer())
config.register_player(name="HeuristicPlayer2", algorithm=HeuristicPlayer())

config.register_player(name="RandomPlayer1", algorithm=RandomPlayer())
config.register_player(name="RandomPlayer2", algorithm=RandomPlayer())

game_result = start_poker(config, verbose=1)

Started the round 1
Street "preflop" started. (community card = [])
"FoldPlayer1" declared "fold:0"
"FoldPlayer2" declared "fold:0"
"HeuristicPlayer1" declared "fold:0"
"HeuristicPlayer2" declared "fold:0"
"RandomPlayer1" declared "raise:799"
"RandomPlayer2" declared "call:799"
"CallPlayer1" declared "call:799"
"CallPlayer2" declared "call:799"
"CallPlayer3" declared "call:799"
Street "flop" started. (community card = ['D8', 'DA', 'H2'])
"CallPlayer2" declared "call:0"
"CallPlayer3" declared "call:0"
"RandomPlayer1" declared "call:0"
"RandomPlayer2" declared "call:0"
"CallPlayer1" declared "call:0"
Street "turn" started. (community card = ['D8', 'DA', 'H2', 'S8'])
"CallPlayer2" declared "call:0"
"CallPlayer3" declared "call:0"
"RandomPlayer1" declared "raise:201"
"RandomPlayer2" declared "raise:416"
"CallPlayer1" declared "call:416"
"CallPlayer2" declared "call:416"
"CallPlayer3" declared "call:416"
"RandomPlayer1" declared "raise:661"
"RandomPlayer2" declared "call:661"
"CallPlayer1" 

In [7]:
game_result

{'players': [{'name': 'CallPlayer1',
   'stack': 3810,
   'state': 'participating',
   'uuid': 'wqaaxcqyjbxlgsrgpysmgo'},
  {'name': 'CallPlayer2',
   'stack': 10,
   'state': 'participating',
   'uuid': 'swxmkfyqdzjfemrjoeqggx'},
  {'name': 'CallPlayer3',
   'stack': 3660,
   'state': 'participating',
   'uuid': 'wvkbuwgaqwphbjbzkrexfi'},
  {'name': 'FoldPlayer1',
   'stack': 1470,
   'state': 'participating',
   'uuid': 'pzinllcteqdecckfxosgvf'},
  {'name': 'FoldPlayer2',
   'stack': 1500,
   'state': 'participating',
   'uuid': 'nfrochlczityfsfzepnnmi'},
  {'name': 'HeuristicPlayer1',
   'stack': 1500,
   'state': 'participating',
   'uuid': 'clhtroynmhymofdrpauicv'},
  {'name': 'HeuristicPlayer2',
   'stack': 1500,
   'state': 'participating',
   'uuid': 'xdcweswaocvsixmvycrofs'},
  {'name': 'RandomPlayer1',
   'stack': 10,
   'state': 'participating',
   'uuid': 'bnyvtjgrucvidumdgsoajd'},
  {'name': 'RandomPlayer2',
   'stack': 40,
   'state': 'participating',
   'uuid': 'wkoylpkg

## Human player from console

In [8]:
import pypokerengine.utils.visualize_utils as U
from pypokerengine.players import BasePokerPlayer

class ConsolePlayer(BasePokerPlayer):
    def __init__(self, input_receiver=None):
        self.input_receiver = input_receiver if input_receiver else self.__gen_raw_input_wrapper()

    def declare_action(self, valid_actions, hole_card, round_state):
        print(U.visualize_declare_action(valid_actions, hole_card, round_state, self.uuid))
        action, amount = self.__receive_action_from_console(valid_actions)
        return action, amount

    def receive_game_start_message(self, game_info):
        print(U.visualize_game_start(game_info, self.uuid))
        self.__wait_until_input()

    def receive_round_start_message(self, round_count, hole_card, seats):
        print(U.visualize_round_start(round_count, hole_card, seats, self.uuid))
        self.__wait_until_input()

    def receive_street_start_message(self, street, round_state):
        print(U.visualize_street_start(street, round_state, self.uuid))
        self.__wait_until_input()

    def receive_game_update_message(self, new_action, round_state):
        print(U.visualize_game_update(new_action, round_state, self.uuid))
        self.__wait_until_input()

    def receive_round_result_message(self, winners, hand_info, round_state):
        print(U.visualize_round_result(winners, hand_info, round_state, self.uuid))
        self.__wait_until_input()

    def __wait_until_input(self):
        input("Enter some key to continue ...")

    def __gen_raw_input_wrapper(self):
        return lambda msg: input(msg)

    def __receive_action_from_console(self, valid_actions):
        flg = self.input_receiver('Enter f(fold), c(call), r(raise).\n >> ')
        if flg in self.__gen_valid_flg(valid_actions):
            if flg == 'f':
                return valid_actions[0]['action'], valid_actions[0]['amount']
            elif flg == 'c':
                return valid_actions[1]['action'], valid_actions[1]['amount']
            elif flg == 'r':
                valid_amounts = valid_actions[2]['amount']
                raise_amount = self.__receive_raise_amount_from_console(valid_amounts['min'], valid_amounts['max'])
                return valid_actions[2]['action'], raise_amount
        else:
            return self.__receive_action_from_console(valid_actions)

    def __gen_valid_flg(self, valid_actions):
        flgs = ['f', 'c']
        is_raise_possible = valid_actions[2]['amount']['min'] != -1
        if is_raise_possible:
            flgs.append('r')
        return flgs

    def __receive_raise_amount_from_console(self, min_amount, max_amount):
        raw_amount = self.input_receiver("valid raise range = [%d, %d]" % (min_amount, max_amount))
        try:
            amount = int(raw_amount)
            if min_amount <= amount and amount <= max_amount:
                return amount
            else:
                print("Invalid raise amount %d. Try again.")
                return self.__receive_raise_amount_from_console(min_amount, max_amount)
        except:
            print("Invalid input received. Try again.")
            return self.__receive_raise_amount_from_console(min_amount, max_amount)

In [9]:
config = setup_config(max_round=1, initial_stack=1500, small_blind_amount=15, summary_file='/dev/null')
config.register_player(name="HeuristicPlayer", algorithm=HeuristicPlayer())
config.register_player(name="ConsolePlayer", algorithm=ConsolePlayer())
game_result = start_poker(config, verbose=0)

-- Game start (UUID = qdpltubuojkdupqivqnfgc) --
-- rule --
  - 2 players game
  - 1 round
  - start stack = 1500
  -        ante = 0
  - small blind = 15
Enter some key to continue ...
-- Round 1 start (UUID = qdpltubuojkdupqivqnfgc) --
-- hole card --
  - ['SK', 'D9']
-- players information --
  - 0 : HeuristicPlayer (mrfdhwthmzmhsjwgobeymq) => state : participating, stack : 1470
  - 1 : ConsolePlayer (qdpltubuojkdupqivqnfgc) => state : participating, stack : 1485
Enter some key to continue ...
-- New street start (UUID = qdpltubuojkdupqivqnfgc) --
-- street --
  - preflop
Enter some key to continue ...
-- Declare your action (UUID = qdpltubuojkdupqivqnfgc) --
-- valid actions --
  - fold
  - call: 30
  - raise: [45, 1500]
-- hole card --
  - ['SK', 'D9']
-- round state --
  - dealer btn : HeuristicPlayer
  - street : preflop
  - community card : []
  - pot : main = 45, side = []
  - players information
    - 0 : HeuristicPlayer (mrfdhwthmzmhsjwgobeymq) => state : participating, stac